In [2]:
import pgmpy
from IPython.display import Image

In [5]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

# Defining individual CPDs.
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6], [0.4]])
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.7], [0.3]])

# The representation of CPD in pgmpy is a bit different than the CPD shown in the above picture. In pgmpy the colums
# are the evidences and rows are the states of the variable. So the grade CPD is represented like this:
#
#    +---------+---------+---------+---------+---------+
#    | diff    | intel_0 | intel_0 | intel_1 | intel_1 |
#    +---------+---------+---------+---------+---------+
#    | intel   | diff_0  | diff_1  | diff_0  | diff_1  |
#    +---------+---------+---------+---------+---------+
#    | grade_0 | 0.3     | 0.05    | 0.9     | 0.5     |
#    +---------+---------+---------+---------+---------+
#    | grade_1 | 0.4     | 0.25    | 0.08    | 0.3     |
#    +---------+---------+---------+---------+---------+
#    | grade_2 | 0.3     | 0.7     | 0.02    | 0.2     |
#    +---------+---------+---------+---------+---------+

cpd_g = TabularCPD(variable='G', variable_card=3,
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])

cpd_l = TabularCPD(variable='L', variable_card=2,
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'],
                   evidence_card=[3])

cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'],
                   evidence_card=[2])

# Associating the CPDs with the network
model.add_cpds(cpd_d, cpd_i, cpd_g, cpd_l, cpd_s)

# check_model checks for the network structure and CPDs and verifies that the CPDs are correctly
# defined and sum to 1.
model.check_model()

True

In [21]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianModel([('D', 'G'), ('I', 'G'), ('D', 'I')])

# Defining individual CPDs.
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6], [0.4]])
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.999, 0.5], [0.001, 0.5]], evidence=['D'], evidence_card=[2])


# The representation of CPD in pgmpy is a bit different than the CPD shown in the above picture. In pgmpy the colums
# are the evidences and rows are the states of the variable. So the grade CPD is represented like this:
#
#    +---------+---------+---------+---------+---------+
#    | diff    | intel_0 | intel_0 | intel_1 | intel_1 |
#    +---------+---------+---------+---------+---------+
#    | intel   | diff_0  | diff_1  | diff_0  | diff_1  |
#    +---------+---------+---------+---------+---------+
#    | grade_0 | 0.3     | 0.05    | 0.9     | 0.5     |
#    +---------+---------+---------+---------+---------+
#    | grade_1 | 0.4     | 0.25    | 0.08    | 0.3     |
#    +---------+---------+---------+---------+---------+
#    | grade_2 | 0.3     | 0.7     | 0.02    | 0.2     |
#    +---------+---------+---------+---------+---------+

cpd_g = TabularCPD(variable='G', variable_card=3,
                   values=[[0.01, 0.05, 0.9,  0.5],
                           [0.0, 0.25, 0.08, 0.3],
                           [0.99, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])

# Associating the CPDs with the network
model.add_cpds(cpd_d, cpd_i, cpd_g)

# check_model checks for the network structure and CPDs and verifies that the CPDs are correctly
# defined and sum to 1.
model.check_model()

True

In [22]:
from pgmpy.inference import BeliefPropagation
bp = BeliefPropagation(model)
bp.calibrate()

In [23]:
bp.map_query(['I', 'G'], evidence={'D': 0})

0it [00:00, ?it/s]


{'I': 0, 'G': 2}

In [27]:
from pgmpy.models import MarkovNetwork
G = MarkovNetwork()
G.add_nodes_from(['D', 'I', 'G'])
G.add_edges_from([('D', 'G'), ('I', 'G'), ('G', 'I')])

True

In [34]:
from pgmpy.factors.discrete import TabularCPD
# Defining individual CPDs.
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6], [0.4]])
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.999, 0.5], [0.001, 0.5]], evidence=['G'], evidence_card=[2])
cpd_g = TabularCPD(variable='G', variable_card=2, values=[[0.999, 0.5, 0.1, 0.3], [0.001, 0.5, 0.9, 0.7]], evidence=['D', 'I'], evidence_card=[2, 2])
G.add_factors(cpd_d, cpd_i, cpd_g)
G.check_model()


ValueError: DiscreteFactor inconsistent with the model.

In [32]:
print(G.get_factors()[0])
print(G.get_factors()[1])
print(G.get_factors()[2])

+------+-----+
| D(0) | 0.6 |
+------+-----+
| D(1) | 0.4 |
+------+-----+
+------+-------+------+
| G    | G(0)  | G(1) |
+------+-------+------+
| I(0) | 0.999 | 0.5  |
+------+-------+------+
| I(1) | 0.001 | 0.5  |
+------+-------+------+
+------+-------+------+------+------+
| D    | D(0)  | D(0) | D(1) | D(1) |
+------+-------+------+------+------+
| I    | I(0)  | I(1) | I(0) | I(1) |
+------+-------+------+------+------+
| G(0) | 0.999 | 0.5  | 0.1  | 0.3  |
+------+-------+------+------+------+
| G(1) | 0.001 | 0.5  | 0.9  | 0.7  |
+------+-------+------+------+------+


In [33]:
bp = BeliefPropagation(G)

ValueError: DiscreteFactor inconsistent with the model.

In [39]:
import numpy as np
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor
G = FactorGraph()
G.add_nodes_from(['a', 'b', 'c'])
phi1 = DiscreteFactor(['a', 'b'], [2, 2], np.array([10, 0.1, 0.1, 0.1]))
phi2 = DiscreteFactor(['b', 'c'], [2, 2], np.random.rand(4))
G.add_factors(phi1, phi2)
G.add_nodes_from([phi1, phi2])
G.add_edges_from([('a', phi1), ('b', phi1),
                  ('b', phi2), ('c', phi2)])
G.check_model()

True

In [40]:
print(G.get_factors()[0])
print(G.get_factors()[1])

+------+------+------------+
| a    | b    |   phi(a,b) |
+======+======+============+
| a(0) | b(0) |    10.0000 |
+------+------+------------+
| a(0) | b(1) |     0.1000 |
+------+------+------------+
| a(1) | b(0) |     0.1000 |
+------+------+------------+
| a(1) | b(1) |     0.1000 |
+------+------+------------+
+------+------+------------+
| b    | c    |   phi(b,c) |
+======+======+============+
| b(0) | c(0) |     0.7868 |
+------+------+------------+
| b(0) | c(1) |     0.9785 |
+------+------+------------+
| b(1) | c(0) |     0.3372 |
+------+------+------------+
| b(1) | c(1) |     0.3788 |
+------+------+------------+


In [41]:
bp = BeliefPropagation(G)
bp.calibrate()

In [42]:
bp.map_query(['a', 'b', 'c'])

0it [00:00, ?it/s]


{'a': 0, 'b': 0, 'c': 1}